In [2]:
import xml.etree.ElementTree as ET, pandas as pd
from tkinter.filedialog import askdirectory, askopenfilename

In [ ]:
xml_file = askopenfilename()
tree = ET.parse(xml_file)
root = tree.getroot()

In [ ]:
LocalDataValues = root[0][4][0]
Ldv = []
for ldv in LocalDataValues:
    Ldv.append(ldv[0].text)

In [ ]:
ServerDataValues = root[0][5][0][4][8]
#Sdv = []
for sdv in ServerDataValues:
    if not(sdv[0].text in Ldv):
        LocalDataValues.attrib.update({'Count':str(int(LocalDataValues.attrib['Count'])+1)})
        new_ldv = LocalDataValues.makeelement('Ldv',{})
        ET.SubElement(new_ldv,'Name',{}).text = sdv[0].text
        ET.SubElement(new_ldv,'DataType',{}).text = sdv[1].text
        ET.SubElement(new_ldv,'NormalSource',{}).text = 'Telemetered'
        LocalDataValues.append(new_ldv)
        #Sdv.append({'Name':sdv[0].text,'DataType':sdv[1].text,'NormalSource':'Telemetered'})

tree.write('ICCP_PDS_2.xml')

In [2]:
CustomerData1 = pd.read_csv(askopenfilename(),sep="|")

c:\Users\rdjaramillo\Anaconda3\envs\Test\lib\site-packages\IPython\core\interactiveshell.py:3135: DtypeWarning: Columns (8,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
CustomerData2 = pd.read_csv(askopenfilename(),sep="|")

In [42]:
s_ini = """<?xml version="1.0" encoding="utf-8"?>
<ArrayOfScript xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">"""
## s.format(id,Alias, AorGroup, NameSignal, Generador)
s = """ 
  <Script>
    <Name>SourceStatus{}</Name>
    <Description />
    <Resource>1734448806493552640</Resource>
    <Properties>
      <UpdateProperty>
        <PropertyID>1152921504606848519</PropertyID>
        <Operation>{}</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1733885856537641737</PropertyID>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606850057</PropertyID>
        <Operation>{}</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606847748</PropertyID>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606848775</PropertyID>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606849289</PropertyID>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606850314</PropertyID>
        <Operation>Manual</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1733885856537641482</PropertyID>
        <Operation>ReadWrite</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1729382256910272010</PropertyID>
        <Operation>Default</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606912257</PropertyID>
        <Operation>True</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606847492</PropertyID>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1729382256910271242</PropertyID>
        <Operation>Status</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1734448806493553673</PropertyID>
        <Operation>751820113765728281</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606848007</PropertyID>
        <Operation>{}</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1734448806493553923</PropertyID>
        <Operation>2</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1729382256910270986</PropertyID>
        <Operation>ABC</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1729382256910271497</PropertyID>
        <Operation>{}</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1729382256910271754</PropertyID>
        <Operation>None</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1734448806493553418</PropertyID>
        <Operation>SourceStatus</Operation>
      </UpdateProperty>
      <UpdateProperty>
        <PropertyID>1152921504606850580</PropertyID>
      </UpdateProperty>
    </Properties>
    <Where />
    <Action>Insert</Action>
    <TimeOfCreated>18:01:55 25/06/2022</TimeOfCreated>
  </Script>
"""
s_fin = "</ArrayOfScript>"

In [3]:
import pyodbc

In [4]:
#conn_prod = pyodbc.connect('Driver={SQL Server}; Server=10.241.109.41,20010\\OASYSHDB;Database=EPSA_Reporting;UID=epsareportes; PWD=Epsa.2020!;')
conn_dev = pyodbc.connect('Driver={SQL Server}; Server=10.241.114.12,20010\\OASYSHDB;Database=ADMS_QueryEngine;UID=Epsareportes; PWD=cmXoasys2;')

In [7]:
Generators = pd.read_sql_query("Select S.customId as SubCID, SM.gid as GeneratorGid, SM.aorGroup as AorGroup, SM.customId as Alias from ADMS_QueryEngine.DMS.SynchronousMachine as SM join ADMS_QueryEngine.DMS.Substation as S ON SM.container = S.gid where SM.islandOperation = 0",conn_dev,coerce_float=False)

In [35]:
Generators['NameSignal'] = Generators.SubCID.str.split('-|:',expand=True)[2] + "_SourceStatusSignal_"+ Generators.Alias.str.rsplit('_',1,expand=True)[1] #Generators.Alias.str.rsplit('-|_',expand=True)


In [43]:
Generators['NameSignal'] = Generators['NameSignal'].fillna("SourceStatus")
Generators['Alias'] = 'SourceStatus para ' + Generators.Alias


In [50]:
Generators
scripts = ""
with open('SourceStatusSignal.xml','w+') as file:
    for i,x in Generators.iterrows():
        scripts += s.format(i,x.Alias, x.AorGroup, x.NameSignal, x.GeneratorGid) 

    text = s_ini+scripts+s_fin
    file.write(text)       
file.close()